<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2*

# Sprint Challenge - Neural Network Foundations

Table of Problems

1. [Defining Neural Networks](#Q1)
2. [Chocolate Gummy Bears](#Q2)
    - Perceptron
    - Multilayer Perceptron
4. [Keras MMP](#Q3)

<a id="Q1"></a>
## 1. Define the following terms:

- **Neuron:** A neuron is the fundamental building block of a neural network. It consists of a set of weights (used to generate a weighted sum of inputs) and an activation, or transfer, function (run on the aforementioned weighted sum to produce an output).

- **Input Layer:** Neurons in the first, or input, layer of a neural network contain a number weights equal to the number of external inputs, bias term included.

- **Hidden Layer:** Every neural network does not contain a hidden layer, although most do, and this is necessary for complete classification of problems that are not linearly separable. Nodes in a hidden layer contain a number of weights equal to the number of nodes in the previous layer, plus an optional bias term. ReLU activation functions are very commonly used in hidden layers.

- **Output Layer:** Neurons in the output layer of a neural network contain a number of weights equal to the number of neurons in the preceding layer, plus an optional bias term. Many networks have only a single output neuron, although some may have more. 

- **Activation:** The activation function of a neuron transforms the weighted sum it calculates into an output (generally with some given set of properties). Common activation functions include the sigmoid and tanh functions, as well as the ReLU family of functions. The identity function can also be considered an activation function!

- **Backpropagation:** Just as a neural network propagates information about its inputs *forward* through its layers, so it propagates information about the error as compared with a target output *backward* through those same layers. This allows the weights in those layers to be updated little by little, gradually reducing the error produced by the network.

## 2. Chocolate Gummy Bears <a id="Q2"></a>

Right now, you're probably thinking, "yuck, who the hell would eat that?". Great question. Your candy company wants to know too. And you thought I was kidding about the [Chocolate Gummy Bears](https://nuts.com/chocolatessweets/gummies/gummy-bears/milk-gummy-bears.html?utm_source=google&utm_medium=cpc&adpos=1o1&gclid=Cj0KCQjwrfvsBRD7ARIsAKuDvMOZrysDku3jGuWaDqf9TrV3x5JLXt1eqnVhN0KM6fMcbA1nod3h8AwaAvWwEALw_wcB). 

Let's assume that a candy company has gone out and collected information on the types of Halloween candy kids ate. Our candy company wants to predict the eating behavior of witches, warlocks, and ghosts -- aka costumed kids. They shared a sample dataset with us. Each row represents a piece of candy that a costumed child was presented with during "trick" or "treat". We know if the candy was `chocolate` (or not chocolate) or `gummy` (or not gummy). Your goal is to predict if the costumed kid `ate` the piece of candy. 

If both chocolate and gummy equal one, you've got a chocolate gummy bear on your hands!?!?!
![Chocolate Gummy Bear](https://ed910ae2d60f0d25bcb8-80550f96b5feb12604f4f720bfefb46d.ssl.cf1.rackcdn.com/3fb630c04435b7b5-2leZuM7_-zoom.jpg)

In [1]:
# Import libraries.
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
# Set random seed for reproducibility.
np.random.seed(0)

In [3]:
# Load data.
candy = pd.read_csv('chocolate_gummy_bears.csv')

In [4]:
# Check data shape.
candy.shape

(10000, 3)

In [5]:
# Preview data.
candy.head()

,chocolate,gummy,ate
0,0,1,1
1,1,0,1
2,0,1,1
3,0,0,0
4,1,1,0


In [6]:
# Candies that are neither chocolate nor gummy.
# Mostly not eaten (94.4%).
candy[candy['chocolate'] + candy['gummy'] == 0]['ate'].value_counts()

0    2377
1     141
Name: ate, dtype: int64

In [7]:
# Candies that are gummy but not chocolate.
# Mostly eaten (94.8%).
candy[(candy['chocolate'] == 0) & (candy['gummy'] == 1)]['ate'].value_counts()

1    2360
0     131
Name: ate, dtype: int64

In [8]:
# Candies that are chocolate but not gummy.
# Mostly eaten (94.8%).
candy[(candy['chocolate'] == 1) & (candy['gummy'] == 0)]['ate'].value_counts()

1    2359
0     130
Name: ate, dtype: int64

In [9]:
# Candies that are both chocolate and gummy.
# Mostly not eaten (94.4%).
candy[candy['chocolate'] + candy['gummy'] == 2]['ate'].value_counts()

0    2362
1     140
Name: ate, dtype: int64

In [10]:
# Do not do any feature engineering (xor function).
xor_candy = (candy['chocolate'] + candy['gummy']) % 2
(xor_candy == candy['ate']).value_counts()

True     9458
False     542
dtype: int64

In [11]:
# Do not normalize any feature engineering (it's just moving
# the decimal point anyway, since we have an even 10k examples.
#
# (Exclusive or accuracy 94.6%.)
(xor_candy == candy['ate']).value_counts(normalize='True')

True     0.9458
False    0.0542
dtype: float64

### Perceptron

To make predictions on the `candy` dataframe. Build and train a Perceptron using numpy. Your target column is `ate` and your features: `chocolate` and `gummy`. Do not do any feature engineering. :P

Once you've trained your model, report your accuracy. You will not be able to achieve more than ~50% with the simple perceptron. Explain why you could not achieve a higher accuracy with the *simple perceptron* architecture, because it's possible to achieve ~95% accuracy on this dataset. Provide your answer in markdown (and *optional* data analysis code) after your perceptron implementation. 

In [12]:
# Define input matrix and target output.
X = candy[['chocolate', 'gummy']].values
y = candy['ate'].values

In [13]:
class Perceptron():
  """
  The perceptron class from our module one assignment.
  """

  def __init__(self,
               activation='sigmoid',
               learning_rate=0.01,
               max_epochs=1000):
    self.activation = activation
    self.learning_rate = learning_rate
    self.max_epochs = max_epochs

  def sigmoid(self, x):
    """
    The sigmoid function.
    """
    return 1 / (1 + np.exp(-x))

  def d_sigmoid(self, x):
    """
    The derivative of the sigmoid function. For gradient descent.
    """
    return self.sigmoid(x) * (1 - self.sigmoid(x))

  def predict(self, inputs):
    """
    Add bias term; generate predictions.
    """
    inputs = np.c_[np.ones(len(inputs)), inputs]
    return self.sigmoid(np.dot(inputs, self.weights))

  def predict_bias(self, inputs_with_bias):
    """
    Generate predictions - used when the inputs already include a bias.
    """
    return self.sigmoid(np.dot(inputs_with_bias, self.weights))

  def train(self, inputs, target):
    """
    Fit the perceptron to a given input dataset and target output.
    """
    # Add bias term.
    input = np.c_[np.ones(len(inputs)), inputs]

    # Adjust dimensionality for single-column target - we don't want to type a
    # bunch of nested brackets.
    target = np.expand_dims(target, axis=1)

    # Initialize random weights in the range (-1, 1).
    self.weights = 2 * np.random.random((len(input[0]), 1)) - 1

    for epoch in range(self.max_epochs):
        prediction = self.predict_bias(input)
        error = prediction - target
        gradient = self.d_sigmoid(np.dot(input, self.weights))
        adjustments = self.learning_rate * error * gradient
        self.weights = self.weights - np.dot(input.T, adjustments)

In [14]:
# Instantiate perceptron.
p = Perceptron(max_epochs=100)

In [15]:
# Train perceptron model.
p.train(X, y)

In [16]:
# Output predictions from the trained model.
perceptron_predictions = np.rint(p.predict(X)).astype(int)

In [17]:
# Summarize predictions (2518 not eaten, 7482 eaten).
np.bincount(np.squeeze(perceptron_predictions))

array([2518, 7482], dtype=int64)

### Simple Perceptron Accuracy

Once you've trained your model, report your accuracy. You will not be able to achieve more than ~50% with the simple perceptron. Explain why you could not achieve a higher accuracy with the simple perceptron architecture, because it's possible to achieve ~95% accuracy on this dataset. Provide your answer in markdown (and optional data analysis code) after your perceptron implementation. 

In [18]:
# Report simple perceptron accuracy.
accuracy_score(y, perceptron_predictions)

0.7236

### Simple Perceptron Accuracy (Explanation)

A simple perceptron can only completely classify a linearly separable function. The exclusive or (xor) function, which this data approximates (94.6% accuracy), is not linearly separable.

100% accuracy will not be achievable, however, even with a multilayer perceptron, because the data is not perfectly clean (it only _approximates_ the xor function - identical inputs do not always produce identical output).

I have no idea what I've done wrong (or non-simple?) to get about 72% accuracy here, though.

### Multilayer Perceptron <a id="Q3"></a>

Using the sample candy dataset, implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. Your Multilayer Perceptron should be implemented in Numpy. 
Your network must have one hidden layer.

Once you've trained your model, report your accuracy. Explain why your MLP's performance is considerably better than your simple perceptron's on the candy dataset. 

P.S. Don't try candy gummy bears. They're disgusting. 

## 3. Keras MMP <a id="Q3"></a>

Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy.
Use the Heart Disease Dataset (binary classification)
Use an appropriate loss function for a binary classification task
Use an appropriate activation function on the final layer of your network.
Train your model using verbose output for ease of grading.
Use GridSearchCV or RandomSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
When hyperparameter tuning, show you work by adding code cells for each new experiment.
Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
You must hyperparameter tune at least 3 parameters in order to get a 3 on this section.

In [19]:
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df = df.sample(frac=1)
print(df.shape)
df.head()

(303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
63,41,1,1,135,203,0,1,132,0,0.0,1,0,1,1
247,66,1,1,160,246,0,1,120,1,0.0,1,3,1,0
293,67,1,2,152,212,0,0,150,0,0.8,1,0,3,0
124,39,0,2,94,199,0,1,179,0,0.0,2,0,2,1
188,50,1,2,140,233,0,1,163,0,0.6,1,1,3,0
